In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("devices_20210129_1621.csv", usecols = ['mdt_id','vehicle_id', 'list_of_app', 'rpt_time']) 

#Manually filter last report by latest date and remove the duplicated entries that has an older date
df['rpt_time'] =pd.to_datetime(df.rpt_time)
df.sort_values('rpt_time', ascending = False , inplace = True)

#trim vehicle plate whitespace
df['vehicle_id'] = df.vehicle_id.str.replace(' ', '')

df.set_index('vehicle_id')

#drop duplicated plate number
df.drop_duplicates(subset ="vehicle_id", 
                     keep = "first", inplace = True) 

#import vehicle master list
df_v = pd.read_csv("SMRT-master.csv", index_col = 'vehicle_id') 

df_smrt = pd.merge(df,  
                     df_v,  
                     on ='vehicle_id',  
                     how ='right') 
df_smrt


,mdt_id,vehicle_id,list_of_app,rpt_time,Hired Status
0,SHB786K ID,SHB786K,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:35:19,Hired
1,864403045571834,SHF184B,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:29:55,Hired
2,861075023920630,SHC4533S,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:17:08,Unhired
3,861075024188252,SHB5734C,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 14:58:44,Hired
4,861075023927445,SHC4759H,"com.ucast.blackbox:1.0.1014.201027_RC,com.grab...",2021-12-01 14:18:28,Hired
...,...,...,...,...,...
1795,NaN,SHF484K,NaN,NaT,Unhired
1796,NaN,SHF493J,NaN,NaT,Unhired
1797,NaN,SHF497Z,NaN,NaT,Unhired
1798,NaN,SHF498X,NaN,NaT,Unhired


In [3]:
df_smrt.dtypes


mdt_id                  object
vehicle_id              object
list_of_app             object
rpt_time        datetime64[ns]
Hired Status            object
dtype: object

In [4]:
#add meter status
df_smrt['meter'] = df_smrt['list_of_app'].str.contains('meter.firmware::610')
df_smrt

,mdt_id,vehicle_id,list_of_app,rpt_time,Hired Status,meter
0,SHB786K ID,SHB786K,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:35:19,Hired,False
1,864403045571834,SHF184B,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:29:55,Hired,True
2,861075023920630,SHC4533S,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:17:08,Unhired,True
3,861075024188252,SHB5734C,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 14:58:44,Hired,True
4,861075023927445,SHC4759H,"com.ucast.blackbox:1.0.1014.201027_RC,com.grab...",2021-12-01 14:18:28,Hired,True
...,...,...,...,...,...,...
1795,NaN,SHF484K,NaN,NaT,Unhired,NaN
1796,NaN,SHF493J,NaN,NaT,Unhired,NaN
1797,NaN,SHF497Z,NaN,NaT,Unhired,NaN
1798,NaN,SHF498X,NaN,NaT,Unhired,NaN


In [5]:
#loop through the df and extract only the vehicle and meter/tariff firmware data. exclude the list of software
df_smrt['tariff'] = 0

for i in df_smrt.index:
    if (df_smrt['meter'][i] == True):   
        df_smrt['meter'][i] = df_smrt['list_of_app'][i].split(',')[-2]
        df_smrt['tariff'][i] = df_smrt['list_of_app'][i].split(',')[-1]
        

df_smrt.to_csv('tariff.csv')
df_smrt


<ipython-input-5-fabad2573775>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_smrt['meter'][i] = df_smrt['list_of_app'][i].split(',')[-2]
<ipython-input-5-fabad2573775>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_smrt['tariff'][i] = df_smrt['list_of_app'][i].split(',')[-1]
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer

,mdt_id,vehicle_id,list_of_app,rpt_time,Hired Status,meter,tariff
0,SHB786K ID,SHB786K,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:35:19,Hired,False,0
1,864403045571834,SHF184B,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:29:55,Hired,meter.firmware::610,tariff.firmware::24635
2,861075023920630,SHC4533S,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 16:17:08,Unhired,meter.firmware::610,tariff.firmware::24635
3,861075024188252,SHB5734C,"com.ucast.blackbox:1.0.1028.201216_RC,com.grab...",2021-12-01 14:58:44,Hired,meter.firmware::610,tariff.firmware::24635
4,861075023927445,SHC4759H,"com.ucast.blackbox:1.0.1014.201027_RC,com.grab...",2021-12-01 14:18:28,Hired,meter.firmware::610,tariff.firmware::24635
...,...,...,...,...,...,...,...
1795,NaN,SHF484K,NaN,NaT,Unhired,NaN,0
1796,NaN,SHF493J,NaN,NaT,Unhired,NaN,0
1797,NaN,SHF497Z,NaN,NaT,Unhired,NaN,0
1798,NaN,SHF498X,NaN,NaT,Unhired,NaN,0


In [ ]:
~(df_smrt['list_of_app'].str.contains('com.ucast.blackbox:1.0.1028.201216_RC', na=False)) & (df_smrt['Hired Status'] == 'Hired') & ~(df_smrt['list_of_app'].str.contains('com.ste.itsd.fms.agilmdmv2:1.0.15', na=False)) & (df_smrt['list_of_app'].str.contains('com.ste.itsd.fms.agilmdm:1.0.7', na=False))

In [ ]:
#filter
df_a = df_smrt[(df_smrt['list_of_app'].str.contains('tariff.firmware', na=False))]

#new dict to store just vehicle and meter/tariff data
vehicle = {}
for i in df_a.index:
    if df_a['list_of_app'][i] == None:
        vehicle[df_a['vehicle_id'][i]] = 'NA'
    else:
        vehicle[df_a['vehicle_id'][i]] = df_a['list_of_app'][i].split(',')[-2:]


df_tariff = pd.DataFrame.from_dict(vehicle, orient='index');
df_tariff
#df_tariff.to_csv('tariff.csv')


In [ ]:
df_a